<a href="https://colab.research.google.com/github/Drew128/Final_Solution_of_12_balls_problem/blob/master/Final_Solution_of_12_balls_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Final Solution of 12 balls problem
There is something like an interview task that I could not solve on a piece of paper. That`s why I want to use some greedy method, brute force, etc.

So, there are 12 balls, one of which has a different weight. There are also scales with bowls and 3 weighings.

In [0]:
from collections import Counter
import math
import numpy as np
from collections import defaultdict

It looks like we have 24 options of the location of the anomalous ball, considering that we don't know it is easier or heavier than the others.

In [2]:
matrix = []
for sign in (-1, 1):
    for position in range(12):
        matrix.append([sign if i == position else 0 for i in range(12)])

variants = np.array(matrix)
[[i, v] for i, v in enumerate(matrix)]

[[0, [-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [1, [0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [2, [0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [3, [0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0]],
 [4, [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0]],
 [5, [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0]],
 [6, [0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0]],
 [7, [0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0]],
 [8, [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0]],
 [9, [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0]],
 [10, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0]],
 [11, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]],
 [12, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [13, [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [14, [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [15, [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
 [16, [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]],
 [17, [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],
 [18, [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]],
 [19, [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [20, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],
 [21, [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]],
 [22, [0

Let's try to assess whether this problem has a solution.

Scales with bowls give 3 different answers at each step: ['<', '>', '=']

Thus, the decision tree will have 3 ^ 3 = 27 options, for deciding 24 options for the location of the ball should be enough.

In [0]:
def scales(subset1, subset2, option):
    if sum(variants[option][subset1]) < sum(variants[option][subset2]):
        return '<'
    if sum(variants[option][subset1]) > sum(variants[option][subset2]):
        return '>'
    if sum(variants[option][subset1]) == sum(variants[option][subset2]):
        return '='

In [6]:
# Check how the scales works for the option #18 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

scales([0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], 18)

'<'

The problem boils down to finding such a partitioning on the scales that gives maximum information at every step.

In other words, it will split the sample as uniformly as possible between ['<', '>', '=']

The partition function takes a split into groups (subsamples of ball indices) in the input and returns a some estimate of uniformity and the distribution of variants between signs when weighing.

In [0]:
def partition(dist):
      c = Counter()
      for i in range(24):
          sign = scales(*dist, i)
          c.update(sign)
      uniformity_estimation = math.fabs(c['<']-c['='])+math.fabs(c['=']-c['>'])+math.fabs(c['<']-c['>'])
      return uniformity_estimation, c

In [8]:
partition([[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11]])

(24.0, Counter({'<': 12, '>': 12}))

The same function, but receives a list of partitions by the bowls at the input and returns options in each branch of the decision tree

In [0]:
def partitioning(distributions, options_left=range(24), estimation=range(37)):
    for dist in distributions:
        c = Counter()
        v = defaultdict(list)
        for i in range(24):
            if i not in options_left: continue
            sign = scales(*dist, i)
            c.update(sign)
            v[sign].append(i)
        uniformity_estimation = math.fabs(c['<']-c['='])+math.fabs(c['=']-c['>'])+math.fabs(c['<']-c['>'])
        if uniformity_estimation in estimation:
            return uniformity_estimation, c, v, dist 

In [10]:
partitioning([[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11]]])


(24.0,
 Counter({'<': 12, '>': 12}),
 defaultdict(list,
             {'<': [0, 1, 2, 3, 4, 5, 18, 19, 20, 21, 22, 23],
              '>': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]}),
 [[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11]])

Before the first weighing, we do not know the state of the balls.

Therefore, we are only interested in the size of the samples for choosing the best bowl division.

Let's try different sample sizes.

In [12]:
print(" # 6", partitioning([[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11]]]))
print(" # 5", partitioning([[[0, 1, 2, 3, 4],       [7, 8, 9, 10, 11]]]))
print(" # 4", partitioning([[[0, 1, 2, 3],             [8, 9, 10, 11]]]))
print(" # 3", partitioning([[[0, 1, 2],                   [9, 10, 11]]]))
print(" # 2", partitioning([[[0, 1],                         [10, 11]]]))
print(" # 1", partitioning([[[0],                                [11]]]))



 # 6 (24.0, Counter({'<': 12, '>': 12}), defaultdict(<class 'list'>, {'<': [0, 1, 2, 3, 4, 5, 18, 19, 20, 21, 22, 23], '>': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]}), [[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11]])
 # 5 (12.0, Counter({'<': 10, '>': 10, '=': 4}), defaultdict(<class 'list'>, {'<': [0, 1, 2, 3, 4, 19, 20, 21, 22, 23], '=': [5, 6, 17, 18], '>': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}), [[0, 1, 2, 3, 4], [7, 8, 9, 10, 11]])
 # 4 (0.0, Counter({'<': 8, '=': 8, '>': 8}), defaultdict(<class 'list'>, {'<': [0, 1, 2, 3, 20, 21, 22, 23], '=': [4, 5, 6, 7, 16, 17, 18, 19], '>': [8, 9, 10, 11, 12, 13, 14, 15]}), [[0, 1, 2, 3], [8, 9, 10, 11]])
 # 3 (12.0, Counter({'=': 12, '<': 6, '>': 6}), defaultdict(<class 'list'>, {'<': [0, 1, 2, 21, 22, 23], '=': [3, 4, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20], '>': [9, 10, 11, 12, 13, 14]}), [[0, 1, 2], [9, 10, 11]])
 # 2 (24.0, Counter({'=': 16, '<': 4, '>': 4}), defaultdict(<class 'list'>, {'<': [0, 1, 22, 23], '=': [2, 3, 4, 5, 6, 7, 8, 9, 14,

It`s seems like sample size = 4 have a good estimation

#It is brute force time
Let's get ALL the samples with size 4

In [13]:
combinations = []
for i1 in range(12):
    for i2 in range(12):
        for i3 in range(12):
            for i4 in range(12):
                if len(set([i1, i2, i3, i4])) == 4 and sorted([i1, i2, i3, i4]) not in combinations:
                    combinations.append(sorted([i1, i2, i3, i4]))

[print(each) for each in combinations[-3:]]                    
print(len(combinations))

[7, 8, 10, 11]
[7, 9, 10, 11]
[8, 9, 10, 11]
495


...and all the options of two samples with size 4

In [14]:
all_4_and_4_samples_option = []
for a in combinations:
    for b in combinations:
        if len(set(a + b)) == 8:
            all_4_and_4_samples_option.append([a, b])

[print(each) for each in all_4_and_4_samples_option[-3:]]
print(len(all_4_and_4_samples_option))

[[8, 9, 10, 11], [3, 4, 6, 7]]
[[8, 9, 10, 11], [3, 5, 6, 7]]
[[8, 9, 10, 11], [4, 5, 6, 7]]
34650


Now we will substitute all the partitioning options, stopping when the required quality is achieved. Then we select the splits in the next step ...

In [15]:
uniformity_estimation, counter1, options_by_signs1, used_distribution1 = partitioning(all_4_and_4_samples_option, estimation=range(3))
for sign1 in options_by_signs1:
    print(used_distribution1[0], sign1, used_distribution1[1], 'options_left=', options_by_signs1[sign1])
    uniformity_estimation, counter2, options_by_signs2, used_distribution2 = partitioning(all_4_and_4_samples_option, options_left = options_by_signs1[sign1], estimation=range(3))
    for sign2 in options_by_signs2:
        print('\t', used_distribution2[0], sign2, used_distribution2[1], 'options_left=', options_by_signs2[sign2])
        uniformity_estimation, counter3, options_by_signs3, used_distribution3 = partitioning(all_4_and_4_samples_option, options_left = options_by_signs2[sign2], estimation=range(3))
        for sign3 in options_by_signs3:
            print('\t\t', used_distribution3[0], sign3, used_distribution3[1], 'options_left=', options_by_signs3[sign3], ' <------', variants[options_by_signs3[sign3]])


[0, 1, 2, 3] < [4, 5, 6, 7] options_left= [0, 1, 2, 3, 16, 17, 18, 19]
	 [0, 1, 2, 4] < [3, 8, 9, 10] options_left= [0, 1, 2]
		 [0, 3, 4, 5] < [1, 6, 7, 8] options_left= [0]  <------ [[-1  0  0  0  0  0  0  0  0  0  0  0]]
		 [0, 3, 4, 5] > [1, 6, 7, 8] options_left= [1]  <------ [[ 0 -1  0  0  0  0  0  0  0  0  0  0]]
		 [0, 3, 4, 5] = [1, 6, 7, 8] options_left= [2]  <------ [[ 0  0 -1  0  0  0  0  0  0  0  0  0]]
	 [0, 1, 2, 4] > [3, 8, 9, 10] options_left= [3, 16]
		 [0, 1, 2, 3] < [5, 6, 7, 8] options_left= [3]  <------ [[ 0  0  0 -1  0  0  0  0  0  0  0  0]]
		 [0, 1, 2, 3] = [5, 6, 7, 8] options_left= [16]  <------ [[0 0 0 0 1 0 0 0 0 0 0 0]]
	 [0, 1, 2, 4] = [3, 8, 9, 10] options_left= [17, 18, 19]
		 [0, 1, 2, 5] > [3, 4, 6, 8] options_left= [17]  <------ [[0 0 0 0 0 1 0 0 0 0 0 0]]
		 [0, 1, 2, 5] < [3, 4, 6, 8] options_left= [18]  <------ [[0 0 0 0 0 0 1 0 0 0 0 0]]
		 [0, 1, 2, 5] = [3, 4, 6, 8] options_left= [19]  <------ [[0 0 0 0 0 0 0 1 0 0 0 0]]
[0, 1, 2, 3] > [4, 5, 6